# Import Event Log

In [3]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.conversion.log import converter as log_converter

if __name__ == "__main__":
    # Read the CSV file
    dataframe_log = pd.read_csv('../../data/logs/gigantic_log.csv', sep=',')

    # Format the dataframe
    dataframe_log = pm4py.format_dataframe(
        dataframe_log,
        case_id='case:concept:name',
        activity_key='concept:name',
        timestamp_key='time:timestamp'
    )

    # Convert the dataframe to event log
    log = log_converter.apply(dataframe_log)

/var/folders/s_/ch_w_j2d0sqf6dbdc0_224m40000gq/T/ipykernel_16995/1982556689.py:11: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  dataframe_log = pm4py.format_dataframe(


# Drop unnessary columns

In [4]:
dataframe_log = dataframe_log.drop(columns=['timestamp'])
dataframe_log = dataframe_log.drop(columns=['case:concept:name'])
dataframe_log = dataframe_log.drop(columns=['@@index'])

In [5]:
from sklearn.preprocessing import StandardScaler

# Convert to datetime format
dataframe_log['time:timestamp'] = pd.to_datetime(dataframe_log['time:timestamp'])

# Calculate elapsed time since the start of each case
dataframe_log['start_time'] = dataframe_log.groupby('@@case_index')['time:timestamp'].transform('min')
dataframe_log['elapsed_time'] = (dataframe_log['time:timestamp'] - dataframe_log['start_time']).dt.total_seconds()

# Normalize the elapsed time in minutes
scaler = StandardScaler()
dataframe_log['standardized_elapsed_time'] = scaler.fit_transform(dataframe_log[['elapsed_time']])

dataframe_log = dataframe_log.drop(columns=['start_time'])
dataframe_log = dataframe_log.drop(columns=['elapsed_time'])
dataframe_log = dataframe_log.drop(columns=['time:timestamp'])

In [6]:
dataframe_log

,concept:name,case:label,org:resource,@@case_index,standardized_elapsed_time
0,Random activity 47,Insert,Random user 44,0,-0.021690
1,Activity A,Insert,Micheline,0,-0.021446
2,Activity T,Insert,Steve,0,-0.021201
3,Activity AC,Insert,Ling,0,-0.020712
4,Activity AE,Insert,Amanda,0,-0.020468
...,...,...,...,...,...
29824,Activity P,normal,Hoyt,4999,-0.020468
29825,Activity M,normal,Maryellen,4999,-0.020224
29826,Activity D,normal,Shandi,4999,-0.019979
29827,Activity S,normal,Chan,4999,-0.019491


# Preprocess

In [7]:
codes, uniques = pd.factorize(dataframe_log['org:resource'])
dataframe_log['org:resource'] = codes + 1

In [8]:
codes, uniques = pd.factorize(dataframe_log['concept:name'])
dataframe_log['concept:name'] = codes + 1

In [9]:
dataframe_log

,concept:name,case:label,org:resource,@@case_index,standardized_elapsed_time
0,1,Insert,1,0,-0.021690
1,2,Insert,2,0,-0.021446
2,3,Insert,3,0,-0.021201
3,4,Insert,4,0,-0.020712
4,5,Insert,5,0,-0.020468
...,...,...,...,...,...
29824,25,normal,30,4999,-0.020468
29825,12,normal,17,4999,-0.020224
29826,13,normal,18,4999,-0.019979
29827,14,normal,19,4999,-0.019491


# Generate Prefixes

In [10]:
df_resource = dataframe_log[['org:resource', '@@case_index']]
df_activity = dataframe_log[['concept:name', '@@case_index']]
df_timestamp = dataframe_log[['standardized_elapsed_time', '@@case_index']]

In [11]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_prefix_windows(df, case_id_column='@@case_index', max_len=None):
    windows = []
    targets = []
    case_indices = []
    
    for case_id in df[case_id_column].unique():
        case_data = df[df[case_id_column] == case_id].drop(columns=[case_id_column]).to_numpy()
        
        # Optional: Make sure to sort the case data if there's an implicit order (e.g., by timestamps)
        # case_data = case_data.sort_values(by='timestamp_column').to_numpy()  # Uncomment and adjust if needed
        
        for i in range(1, len(case_data)):
            window = case_data[:i]
            target = case_data[i]
            windows.append(window)
            targets.append(target)
            case_indices.append(case_id)
    
    if max_len is None:
        max_len = max(len(window) for window in windows)
    
    # Pad sequences
    windows_padded = pad_sequences(windows, maxlen=max_len, padding='post', dtype='float32')
    
    # Convert targets to numpy array
    targets_array = np.array(targets, dtype='float32')
    case_indices_array = np.array(case_indices)
    
    return windows_padded, targets_array, case_indices_array


2024-07-02 17:35:18.329568: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
windows_resource, targets_resource, case_indices = generate_prefix_windows(df_resource)
windows_activity, targets_activity, case_indices = generate_prefix_windows(df_activity)
windows_timestamp, targets_timestamp, case_indices = generate_prefix_windows(df_timestamp)

# GRU

### Architecture

- Separate Inputs for Each Attribute
- Each attribute is passed through an embedding layer
- Each attribute has its corresponding GRU encoder
- Selective Concatenation: After encoding, the outputs of these GRU layers are concatenated. However, this concatenation is selective, meaning it is structured in a way that prepares the data for effective synthesis without leaking information from the future (next event attributes)
- Decoder GRUs: Integrated Decoding: Post-concatenation, the combined attributes are processed through decoder GRU layers. These layers are tasked with integrating the data from different attributes and preparing it for final prediction. This step is where BINet v3 distinguishes itself by effectively using the interdependencies between different attributes to enhance prediction accuracy.
- Output Layer: Softmax Output for Each Attribute: For each attribute of the next event, a softmax layer predicts a probability distribution over all possible values. This allows the model to output the most likely next event and its attributes based on the learned dependencies and the history encoded by the GRUs.
- E: maximum case length
- We train BINet with a GRU size of 2E (two times the maximum case length)
- on mini batches of size 500 for 20 epochs

In [13]:
# Group by the @@case_index column and count the rows in each group
case_lengths = dataframe_log.groupby('@@case_index').size()

# Find the maximum value among the case lengths
E = case_lengths.max()

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Embedding, Dense, Dropout, Concatenate, BatchNormalization

def create_binetv3(num_activities, num_resources, embedding_dim, gru_units, dropout_rate):
    # Input layers for each attribute
    input_activity = Input(shape=(None,), name='activity_input')
    input_resource = Input(shape=(None,), name='resource_input')
    input_timestamp = Input(shape=(None, 1), name='timestamp_input')

    # Embedding layers for categorical attributes
    embedding_activity = Embedding(input_dim=num_activities + 1, output_dim=embedding_dim, mask_zero=True, name='activity_embedding')(input_activity)
    embedding_resource = Embedding(input_dim=num_resources + 1, output_dim=embedding_dim, mask_zero=True, name='resource_embedding')(input_resource)

    # Encoder GRUs with Batch Normalization for categorical attributes
    encoded_activity = GRU(units=gru_units, return_sequences=True, name='activity_encoder')(embedding_activity)
    bn_activity = BatchNormalization(name='bn_activity')(encoded_activity)
    encoded_resource = GRU(units=gru_units, return_sequences=True, name='resource_encoder')(embedding_resource)
    bn_resource = BatchNormalization(name='bn_resource')(encoded_resource)

    # Encoder GRU with Batch Normalization for continuous attribute
    encoded_timestamp = GRU(units=gru_units, return_sequences=True, name='timestamp_encoder')(input_timestamp)
    bn_timestamp = BatchNormalization(name='bn_timestamp')(encoded_timestamp)

    # Concatenation of encoded outputs
    concatenated = Concatenate(name='concatenate_encodings')([bn_activity, bn_resource, bn_timestamp])

    # Decoder GRU
    decoder_output = GRU(units=gru_units, return_sequences=False, name='decoder_gru')(concatenated)
    dropout_layer = Dropout(rate=dropout_rate, name='dropout')(decoder_output)

    # Output layers for predicting the next event's attributes
    output_activity = Dense(num_activities + 1, activation='softmax', name='output_activity')(dropout_layer)
    output_resource = Dense(num_resources + 1, activation='softmax', name='output_resource')(dropout_layer)
    output_timestamp = Dense(1, activation='linear', name='output_timestamp')(dropout_layer)

    # Building the model
    model = Model(inputs=[input_activity, input_resource, input_timestamp], outputs=[output_activity, output_resource, output_timestamp])
    model.compile(
        optimizer='adam', 
        loss={
            'output_activity': 'categorical_crossentropy', 
            'output_resource': 'categorical_crossentropy',
            'output_timestamp': 'mse'
        },
        metrics={
            'output_activity': ['accuracy'], 
            'output_resource': ['accuracy'],
            'output_timestamp': ['mse']
        }
    )

    return model

# Parameters
gru_units = int(2 * E) 
num_activities = dataframe_log['concept:name'].max()
num_resources = dataframe_log['org:resource'].max()
embedding_dim = 50
dropout_rate = 0.2
model = create_binetv3(num_activities, num_resources, embedding_dim, gru_units, dropout_rate)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 activity_input (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 resource_input (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 activity_embedding (Embedd  (None, None, 50)             7700      ['activity_input[0][0]']      
 ing)                                                                                         

### Data Splitting

In [15]:
from sklearn.model_selection import train_test_split

train_resource, test_resource, train_targets_resource, test_targets_resource = train_test_split(
    windows_resource, targets_resource, test_size=0.3, random_state=42)

train_activity, test_activity, train_targets_activity, test_targets_activity = train_test_split(
    windows_activity, targets_activity, test_size=0.3, random_state=42)

train_timestamp, test_timestamp, train_targets_timestamp, test_targets_timestamp = train_test_split(
    windows_timestamp, targets_timestamp, test_size=0.3, random_state=42)

### Training

In [16]:
from tensorflow.keras.utils import to_categorical

train_targets_resource_cat = to_categorical(train_targets_resource, num_classes=num_resources + 1)
test_targets_resource_cat = to_categorical(test_targets_resource, num_classes=num_resources + 1)

train_targets_activity_cat = to_categorical(train_targets_activity, num_classes=num_activities + 1)
test_targets_activity_cat = to_categorical(test_targets_activity, num_classes=num_activities + 1)

In [17]:
history = model.fit(
    [train_activity, train_resource, train_timestamp],
    [train_targets_activity_cat, train_targets_resource_cat, train_targets_timestamp],
    validation_data=([test_activity, test_resource, test_timestamp], [test_targets_activity_cat, test_targets_resource_cat, test_targets_timestamp]),
    epochs=20,
    batch_size=500
)

Epoch 1/20
35/35 [==============================] - 22s 200ms/step - loss: 11.2048 - output_activity_loss: 4.7461 - output_resource_loss: 4.7867 - output_timestamp_loss: 1.6720 - output_activity_accuracy: 0.1956 - output_resource_accuracy: 0.0728 - output_timestamp_mse: 1.6720 - val_loss: 9.8525 - val_output_activity_loss: 4.8956 - val_output_resource_loss: 4.8520 - val_output_timestamp_loss: 0.1049 - val_output_activity_accuracy: 0.2057 - val_output_resource_accuracy: 0.0411 - val_output_timestamp_mse: 0.1049
Epoch 2/20
35/35 [==============================] - 2s 71ms/step - loss: 9.6213 - output_activity_loss: 3.6792 - output_resource_loss: 4.2167 - output_timestamp_loss: 1.7254 - output_activity_accuracy: 0.3288 - output_resource_accuracy: 0.1622 - output_timestamp_mse: 1.7254 - val_loss: 9.2671 - val_output_activity_loss: 4.5263 - val_output_resource_loss: 4.6394 - val_output_timestamp_loss: 0.1015 - val_output_activity_accuracy: 0.2057 - val_output_resource_accuracy: 0.1152 - val_

In [18]:
# Evaluate the model on the validation set
results = model.evaluate(
    [test_activity, test_resource, test_timestamp],
    [test_targets_activity_cat, test_targets_resource_cat, test_targets_timestamp],
    batch_size=64
)
print(f"Validation Loss: {results[0]}, Validation Accuracy: {results[1]}")

117/117 [==============================] - 1s 6ms/step - loss: 2.5624 - output_activity_loss: 0.5228 - output_resource_loss: 1.9571 - output_timestamp_loss: 0.0825 - output_activity_accuracy: 0.9048 - output_resource_accuracy: 0.4991 - output_timestamp_mse: 0.0825
Validation Loss: 2.5623981952667236, Validation Accuracy: 0.5227868556976318


In [19]:
# Save the model to an H5 file
model.save('binetv3_gigantic.h5')

/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Anomaly Score Computation

- For each event attribute, BINet's softmax layer outputs a probability distribution over possible values
- The anomaly score for a specific attribute value v is calculated by summing all the probabilities from the softmax output that are greater than the probability assigned to v

In [20]:
# Generate predictions for all inputs
predictions = model.predict([windows_activity, windows_resource, windows_timestamp])


# Assuming your model is set to predict resource, activity, and role categories
# Extract predictions for categorical attributes (softmax probabilities)
predictions_activity = predictions[0]  # Resource predictions
predictions_resource = predictions[1]  # Activity predictions

# If you had added 'amount' as a target to be predicted, you would extract its predictions like so:
predictions_timestamp = predictions[2]  # Assuming 'amount' is a regression target and the model is adjusted accordingly

776/776 [==============================] - 9s 5ms/step


In [21]:
import numpy as np

def calculate_anomaly_scores(predictions, targets):
    scores = []
    # Loop through each example in the predictions
    for i in range(predictions.shape[0]):
        actual_prob = predictions[i, targets[i]]  # Extract the probability of the true class using target index
        # Calculate anomaly score as sum of probabilities greater than the probability of the actual value
        anomaly_score = np.sum(predictions[i][predictions[i] > actual_prob])
        scores.append(anomaly_score)

    return scores

In [22]:
anomaly_scores_activity = calculate_anomaly_scores(predictions_activity, targets_activity.astype(int))
anomaly_scores_resource = calculate_anomaly_scores(predictions_resource, targets_resource.astype(int))

In [23]:
def compute_anomaly_scores_continuous(predictions, actuals):
    # Calculate absolute differences
    differences = np.abs(predictions - actuals)
    
    # Normalize to [0, 1] range
    max_diff = np.max(differences)
    normalized_scores = differences / max_diff if max_diff != 0 else differences
    
    return normalized_scores

In [24]:
anomaly_scores_timestamp = compute_anomaly_scores_continuous(predictions_timestamp, targets_timestamp)
anomaly_scores_timestamp = anomaly_scores_timestamp.flatten()
anomaly_scores_timestamp = anomaly_scores_timestamp.tolist()

## Insert missing scores for cases with less than 2 Events

In [25]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
score = pd.DataFrame({'case': case_indices})
score['score_resource'] = anomaly_scores_resource
score['score_activity'] = anomaly_scores_activity
score['score_timestamp'] = anomaly_scores_timestamp

score['case'] = score['case'].astype(int)

score

,case,score_resource,score_activity,score_timestamp
0,0,0.000000,0.0,0.001552
1,0,0.822019,0.0,0.000824
2,0,0.308357,0.0,0.000026
3,0,0.000000,0.0,0.002063
4,0,0.162349,0.0,0.000118
...,...,...,...,...
24824,4999,0.000000,0.0,0.002245
24825,4999,0.000000,0.0,0.000489
24826,4999,0.547130,0.0,0.000878
24827,4999,0.791218,0.0,0.000355


In [26]:
import pandas as pd

def contains_all_values(df, column, end):

    # Generate the set of all values in the specified range
    required_values = set(range(0, end + 1))
    
    # Get the unique values in the specified column
    column_values = set(df[column].unique())
    
    # Find missing values
    missing_values = required_values - column_values
    
    # Print missing values if any
    if missing_values:
        print(f"Missing values: {sorted(missing_values)}")
    
    # Check if all required values are in the column values
    return required_values.issubset(column_values)

end = 4999

result = contains_all_values(score, 'case', end)
print(f"Does the 'case' column contain all values between 0 and {end}? {result}")



Missing values: [42, 100, 200, 206, 266, 288, 358, 502, 727, 756, 777, 838, 945, 998, 1159, 1195, 1196, 1231, 1285, 1297, 1518, 1843, 1862, 1863, 1908, 1927, 1969, 2142, 2205, 2271, 2328, 2338, 2378, 2461, 2859, 2868, 2884, 2956, 3037, 3076, 3122, 3129, 3211, 3226, 3228, 3234, 3236, 3249, 3263, 3317, 3324, 3431, 3446, 3450, 3497, 3531, 3651, 3702, 3712, 3798, 3865, 3914, 4073, 4223, 4257, 4325, 4332, 4342, 4653, 4805, 4843, 4863, 4965, 4984]
Does the 'case' column contain all values between 0 and 4999? False


In [27]:
# Generate the full range of values
full_range = set(range(4999+1))

# Get the existing 'case' values
existing_cases = set(score['case'])

# Identify missing values
missing_cases = full_range - existing_cases

# Create a DataFrame for missing values
missing_df = pd.DataFrame({'case': list(missing_cases)})

# Add other columns with value 1
for col in score.columns:
    if col != 'case':
        missing_df[col] = 1

# Concatenate the original DataFrame with the missing values DataFrame
updated_score = pd.concat([score, missing_df])

# Sort the DataFrame by 'case'
updated_score = updated_score.sort_values(by='case').reset_index(drop=True)

updated_score

,case,score_resource,score_activity,score_timestamp
0,0,0.000000,0.0,0.001552
1,0,0.822019,0.0,0.000824
2,0,0.308357,0.0,0.000026
3,0,0.000000,0.0,0.002063
4,0,0.162349,0.0,0.000118
...,...,...,...,...
24898,4999,0.791218,0.0,0.000355
24899,4999,0.385613,0.0,0.000384
24900,4999,0.000000,0.0,0.000436
24901,4999,0.701260,0.0,0.000583


In [28]:
anomaly_scores_resource = updated_score['score_resource']
anomaly_scores_activity = updated_score['score_activity']
anomaly_scores_timestamp = updated_score['score_timestamp']

### Threshold (lowest plateau)

In [29]:
import numpy as np

def calculate_anomaly_ratio(scores, threshold):
    """
    Calculate the anomaly ratio for a given threshold.
    """
    return np.mean(scores > threshold)

def find_plateaus(scores, epsilon=1e-4, min_plateau_length=10):
    """
    Identify the lowest plateau in the anomaly ratio function and calculate the mean-centered threshold.
    """
    scores = np.array(scores)  # Convert scores to a NumPy array
    sorted_scores = np.sort(scores)
    
    # Remove duplicate values
    unique_thresholds, unique_indices = np.unique(sorted_scores, return_index=True)
    anomaly_ratios = np.array([calculate_anomaly_ratio(scores, t) for t in unique_thresholds])
    
    # Calculate first and second derivatives
    first_derivatives = np.diff(anomaly_ratios) / np.diff(unique_thresholds)
    second_derivatives = np.diff(first_derivatives) / np.diff(unique_thresholds[:-1])
    
    # Identify plateaus where the first derivative is close to zero
    plateau_indices = np.where(np.abs(first_derivatives) < epsilon)[0]
    
    # Group consecutive indices to identify continuous plateaus
    grouped_plateaus = np.split(plateau_indices, np.where(np.diff(plateau_indices) != 1)[0] + 1)
    
    # Filter plateaus based on minimum length
    long_plateaus = [g for g in grouped_plateaus if len(g) >= min_plateau_length]
    
    if long_plateaus:
        # Take the first long plateau and find the mean threshold in this plateau
        first_plateau = long_plateaus[0]
        plateau_thresholds = unique_thresholds[first_plateau]
        return np.mean(plateau_thresholds)
    else:
        # If no plateau is found, return a default value, e.g., the 90th percentile
        percentile_90 = np.percentile(sorted_scores, 90)
        if percentile_90 == 1.0:
            return 0.4
        else:
            return percentile_90

In [30]:
threshold_resource = find_plateaus(anomaly_scores_resource)
threshold_activity = find_plateaus(anomaly_scores_activity)
threshold_timestamp = find_plateaus(anomaly_scores_timestamp)

### Detection

In [31]:
def detect_anomalies(anomaly_scores, threshold):
    labels = [1 if score > threshold else 0 for score in anomaly_scores]
    return labels

In [32]:
# Detect anomalies based on the calculated anomaly scores and thresholds
labels_resource = detect_anomalies(anomaly_scores_resource, threshold_resource)
labels_activity = detect_anomalies(anomaly_scores_activity, threshold_activity)
labels_timestamp = detect_anomalies(anomaly_scores_timestamp, threshold_timestamp)

# Mapping

In [33]:
case_indices = updated_score['case']

In [34]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
mapping = pd.DataFrame({'case': case_indices})
mapping['predicted_resource'] = labels_resource
mapping['predicted_activity'] = labels_activity
mapping['predicted_timestamp'] = labels_timestamp

mapping

,case,predicted_resource,predicted_activity,predicted_timestamp
0,0,0,0,1
1,0,1,0,0
2,0,0,0,0
3,0,0,0,1
4,0,0,0,0
...,...,...,...,...
24898,4999,1,0,0
24899,4999,0,0,0
24900,4999,0,0,0
24901,4999,0,0,0


In [62]:
# Create a boolean DataFrame where each value is True if the value is 1
contains_one = (mapping[['predicted_resource', 'predicted_activity', 'predicted_timestamp']] == 1)

# Group by 'case' and check if there's at least one 'True' in any of the columns
case_prediction = contains_one.groupby(mapping['case']).any().any(axis=1)
case_prediction

case
0        True
1        True
2        True
3        True
4        True
        ...  
4995     True
4996    False
4997     True
4998    False
4999     True
Length: 5000, dtype: bool

In [49]:
# Create a boolean DataFrame where each value is True if the value is 1
contains_one = (mapping[['predicted_activity']] == 1)

# Group by 'case' and check if there's at least one 'True' in any of the columns
case_prediction = contains_one.groupby(mapping['case']).any().any(axis=1)
case_prediction

case
0       False
1       False
2       False
3        True
4       False
        ...  
4995     True
4996    False
4997     True
4998    False
4999    False
Length: 5000, dtype: bool

# Ground Truth

In [63]:
def generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking):
    from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
    from pm4py.algo.conformance.alignments.petri_net import variants
    from pm4py.objects.petri_net.utils import align_utils
    max_events=0
    for trace in log:
        counter=0
        for event in trace:
            counter+=1
        if counter > max_events:
            max_events=counter
    parameters={}
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_SYNC_COST_FUNCTION] = list(map(lambda i: .1*i, range(max_events*2)))
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_TRACE_COST_FUNCTION]=list(map(lambda i: align_utils.STD_MODEL_LOG_MOVE_COST-.1*i, range(max_events*2)))
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking, variant=variants.state_equation_a_star, parameters=parameters)
    return aligned_traces

In [64]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/model/gigantic.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_traces = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/738 [00:00<?, ?it/s]

In [65]:
def extract_conformance_status_by_fitness(aligned_traces):
    conformance_status = []
    for alignment in aligned_traces:
        fitness = alignment['fitness']
        # If the fitness is 1.0, the trace is conforming
        if fitness == 1.0:
            conformance_status.append(1)
        else:
            conformance_status.append(0)
    return conformance_status

# Get the conformance status list from the aligned traces
conformance = extract_conformance_status_by_fitness(aligned_traces)

In [66]:
ground_truth = pd.DataFrame({'conformity': conformance})
ground_truth['predicted'] = case_prediction

# Convert False to 0 and True to 1
ground_truth['predicted'] = [int(value) for value in ground_truth['predicted']]
ground_truth['predicted'] = 1 - ground_truth['predicted']
ground_truth

,conformity,predicted
0,0,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
4995,0,0
4996,0,1
4997,1,0
4998,1,1


# Evaluation

In [67]:
# Calculating TP, TN, FP, FN
TP = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 1)).sum()
TN = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 0)).sum()
FP = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 1)).sum()
FN = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 0)).sum()

In [68]:
# Calculate accuracy
accuracy = (TP + TN) / (TP + TN + FP + FN)
print(f"Accuracy: {accuracy:.3f}")

Accuracy: 0.581


In [69]:
# Calculate f1

precision = TP / (TP + FP)
recall = TP / (TP + FN)

f1 = 2 * ((precision * recall) / (precision + recall))
print(f"F1: {f1:.3f}")

F1: 0.664


### Dev (Non Conform Traces)

In [70]:
# Calculate precision for Dev
precision = TN / (TN + FN)
print(f"Precision: {precision:.3f}")

Precision: 0.316


In [71]:
# Calculate recall for Dev
recall = TN / (TN + FP)
print(f"Recall: {recall:.3f}")

Recall: 0.732


### No Dev (Conform Traces)

In [72]:
# Calculate precision for No Dev
precision = TP / (TP + FP)
print(f"Precision: {precision:.3f}")

Precision: 0.872


In [73]:
# Calculate recall for No Dev
recall = TP / (TP + FN)
print(f"Recall: {recall:.3f}")

Recall: 0.536


### AUC-ROC

In [74]:
import pandas as pd
from sklearn.metrics import roc_auc_score

# Assuming ground_truth is your DataFrame
# Make sure 'conformity' contains actual labels (0 or 1)
# and 'predicted' contains predicted probabilities or scores
auc_roc = roc_auc_score(ground_truth['conformity'], ground_truth['predicted'])
auc_roc

0.6340094323365769